In [25]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_games = pd.read_csv("originales/games.csv")
df_teams = pd.read_csv("originales/teams.csv")
df_teamstats = pd.read_csv("originales/teamstats.csv")
df_leagues = pd.read_csv("originales/leagues.csv")
df_appearances = pd.read_csv("originales/appearances.csv")
df_players = pd.read_csv("originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("originales/shots.csv")



""" df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") """

' df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")\ndf_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")\ndf_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")\ndf_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")\ndf_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")\ndf_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")\ndf_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") '

In [26]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["homeProbability", "drawProbability", "awayProbability"], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)





In [27]:
df2_appearances = pd.read_csv("players/appearances.csv")
df2_club_games = pd.read_csv("players/club_games.csv")
df2_clubs = pd.read_csv("players/clubs.csv")
df2_competitions = pd.read_csv("players/competitions.csv")
df2_games = pd.read_csv("players/games.csv")
df2_player_val= pd.read_csv("players/player_valuations.csv")
df2_players = pd.read_csv("players/players.csv")
df2_players.info()


""" df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") """


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25619 entries, 0 to 25618
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    25619 non-null  int64  
 1   pretty_name                  25619 non-null  object 
 2   club_id                      25619 non-null  int64  
 3   club_pretty_name             25619 non-null  object 
 4   current_club_id              25619 non-null  int64  
 5   country_of_citizenship       25618 non-null  object 
 6   country_of_birth             23903 non-null  object 
 7   date_of_birth                25593 non-null  object 
 8   position                     25619 non-null  object 
 9   sub_position                 22544 non-null  object 
 10  name                         25619 non-null  object 
 11  foot                         23727 non-null  object 
 12  height_in_cm                 25619 non-null  int64  
 13  market_value_in_

' df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")\ndf2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")\ndf2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")\ndf2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")\ndf2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")\ndf2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")\ndf2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") '

In [28]:

top_5_ligas=df2_competitions.loc[(df2_competitions['pretty_name']=='Bundesliga') | (df2_competitions['pretty_name']=='Laliga') | (df2_competitions['pretty_name']=='Premier League') | (df2_competitions['pretty_name']=='Serie A') | (df2_competitions['pretty_name']=='Ligue 1') ]
#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein
df_f_game[['date','hour']]=df_f_game['date'].str.split(' ',1,expand=True)
pd.options.mode.chained_assignment = None  # default='warn'

from fuzzywuzzy import fuzz, process


from thefuzz import process, fuzz

def match(x,df,score):
  match=process.extractOne(x, df,score_cutoff=score,scorer=fuzz.WRatio)
  
  
  if match:
    return match[2]
  else:
    return 2

top_5_ligas['new_id'] = df_teams.loc[top_5_ligas["pretty_name"].apply(lambda x: match(x,df_teams.name,70) ).values,"name"].values
   # See note below

seasons=df_f_game["season"].unique()


df2_games= df2_games[df2_games['season'].isin(seasons)]
df2_games_top5= df2_games[df2_games['competition_id'].isin(top_5_ligas['competition_id'])]



df2_clubs_top5=df2_clubs[df2_clubs['domestic_competition_id'].isin(top_5_ligas['competition_id'])]

df2_clubs_top5['new_id'] = df_teams.loc[df2_clubs_top5['pretty_name'].map(lambda x: match(x,df_teams.name,80) ).values,"teamID"].values



df2_appearances_top5= df2_appearances[df2_appearances['game_id'].isin(df2_games_top5['game_id'])]


df2_players_top5=df2_players[df2_players['current_club_id'].isin(df2_clubs_top5['club_id'])]





df2_players_top5['new_id'] = df_players.loc[df2_players_top5['pretty_name'].apply(lambda x: match(x,df_players.name,70) ).values,"name"].values
df2_players_top5.to_csv("asdfasdfsa.csv")


df2_players_val_top5=df2_player_val[df2_player_val['player_id'].isin(df2_players_top5['player_id'])]






In [29]:
df2_players_top5

,player_id,pretty_name,club_id,club_pretty_name,current_club_id,country_of_citizenship,country_of_birth,date_of_birth,position,sub_position,...,market_value_in_gbp,highest_market_value_in_gbp,agent_name,contract_expiration_date,domestic_competition_id,club_name,image_url,last_season,url,new_id
0,254016,Arthur Delalande,855,Ea Guingamp,855,France,France,1992-05-18,Midfield,Central Midfield,...,NaN,90000.0,NaN,NaN,FR1,ea-guingamp,NaN,2013,https://www.transfermarkt.co.uk/arthur-delalan...,Arthur
1,51053,Daniel Davari,23,Eintracht Braunschweig,23,Iran,Germany,1988-01-06,Goalkeeper,NaN,...,135000.0,1130000.0,NG360,2022-06-30,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/daniel-davari/...,Ari
2,31451,Torsten Oehrl,23,Eintracht Braunschweig,23,Germany,Germany,1986-01-07,Attack,Centre-Forward,...,NaN,1130000.0,NaN,NaN,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/torsten-oehrl/...,Daley Blind
4,30802,Oscar Diaz,3302,Ud Almeria,3302,Spain,Spain,1984-04-24,Attack,Centre-Forward,...,NaN,1080000.0,Pedro Bravo - Consulting,2022-06-30,ES1,ud-almeria,NaN,2013,https://www.transfermarkt.co.uk/oscar-diaz/pro...,Oscar
6,117549,Rodri Rios,3302,Ud Almeria,3302,Spain,Spain,1990-06-06,Attack,Centre-Forward,...,270000.0,1800000.0,NaN,2022-06-30,ES1,ud-almeria,NaN,2013,https://www.transfermarkt.co.uk/rodri-rios/pro...,Rodri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25197,569385,Adil Aouchiche,1158,Fc Lorient,1158,France,France,2002-07-15,Attack,Attacking Midfield,...,3000000.0,10000000.0,Sport Cover,2026-06-30,FR1,fc-lorient,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/adil-aouchiche...,Adil Aouchiche
25201,238626,Laurent Abergel,1158,Fc Lorient,1158,France,France,1993-02-01,Midfield,Defensive Midfield,...,3000000.0,3000000.0,Just 4 player Group,2025-06-30,FR1,fc-lorient,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/laurent-aberge...,Laurent Abergel
25208,496706,Montassar Talbi,1158,Fc Lorient,1158,Tunisia,France,1998-05-26,Defender,Centre-Back,...,4000000.0,4000000.0,Wasserman,2025-06-30,FR1,fc-lorient,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/montassar-talb...,Fontàs
25215,333383,Bonke Innocent,1158,Fc Lorient,1158,Nigeria,Nigeria,1996-01-20,Midfield,Defensive Midfield,...,2000000.0,2000000.0,HCM Sports Management,2025-06-30,FR1,fc-lorient,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/bonke-innocent...,Kevin
